In [158]:
import numpy as np
import pandas as pd
log_df = pd.read_excel("./data/preprocessed_광진구_데이터프레임.xlsx",index_col=0)
log_df

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가_기준년월,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,,
서울특별시 광진구 광장동 102,8.86329,4.91265,0.69315,7.70156,1.38629,15.65986,7.04229,3.71357
서울특별시 광진구 광장동 104,8.86329,4.91265,0.69315,5.14808,1.60944,15.44416,7.06133,3.71357
서울특별시 광진구 광장동 105,8.86329,4.91265,0.69315,6.74382,1.38629,15.46455,6.97354,3.71357
서울특별시 광진구 광장동 106,8.86329,4.91265,0.69315,6.11545,1.38629,15.44416,6.94505,3.71357
서울특별시 광진구 광장동 107,8.86329,4.91265,0.69315,6.00117,1.38629,15.57216,6.91870,3.73767
...,...,...,...,...,...,...,...,...
서울특별시 광진구 화양동 95-5,10.86601,3.78419,3.33220,5.02388,1.38629,15.28477,6.44572,4.31749
서울특별시 광진구 화양동 96-2,10.86601,3.78419,3.43399,5.10595,1.38629,15.28477,6.44572,4.31749
서울특별시 광진구 화양동 97,10.86601,3.78419,3.33220,5.23644,1.38629,15.44377,6.45047,4.30407


### UMAP

In [208]:
def get_umap(df, n_neighbors, min_dist):
    '''
    n_neighbors가 작을수록 지역적 구조가 강조됨
    min_dist: 포인트 간의 거리 지정하기
    '''
    import umap
    clustering_embedding = umap.UMAP(
    n_neighbors=n_neighbors, # 5~ 50 기본값 15
    min_dist=min_dist, # 0.001~ 0.5 기본값 0.1
    n_components=2,
    random_state=42,
    metric = 'cosine'
    ).fit_transform(df.values)
    return clustering_embedding # 100, 0.005

### HDBSCAN 클러스터링

In [164]:
def get_hdbscan(embedding, min_cluster_size, min_samples):
    '''
    min_cluster_size(군집의 최소 크기)와 min_samples(노이즈로 간주되는 최소 점의 수)만 인자로 받아서
    hdbscan진행하기
    '''
    import hdbscan
    labels = hdbscan.HDBSCAN(
    algorithm='best', alpha=1.0, approx_min_span_tree=True,
    gen_min_span_tree=True, leaf_size=40,
    metric='euclidean', min_cluster_size=min_cluster_size, min_samples=min_samples
    ).fit_predict(embedding)
    return labels # 800~1000

In [218]:
def get_all_cluster(embedding,labels):
    # 노이즈 포인트 다 포함
    plt.scatter(embedding[:, 0], embedding[:, 1],
                c=labels,s=0.1, cmap='Spectral')
    plt.show()
def get_noise_cluster(embedding,labels):
    # 노이즈 포인트만
    clustered = (labels >= 0)
    plt.scatter(embedding[~clustered, 0],
                embedding[~clustered, 1],
                color=(0.5, 0.5, 0.5),
                s=0.1,
                alpha=0.5)
    plt.show()
def get_cluster(embedding,labels):
    # 노이즈 포인트 제외
    clustered = (labels >= 0)
    plt.scatter(embedding[clustered, 0],
                embedding[clustered, 1],
                c=labels[clustered],
                s=0.1,
                cmap='Spectral')
    plt.show()

In [49]:
pd.options.display.float_format = '{:.5f}'.format

### MinMax 스케일링 해보기

In [62]:
def get_scaler(df):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaled_df = scaler.fit_transform(df.values)
    scaled_df = pd.DataFrame(data=scaled_df, index=df.index, columns=df.columns)
    return scaled_df

### 로버스트 스케일러 해보기

In [95]:
def get_robust_scaler(df):
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    scaled_df = scaler.fit_transform(df.values)
    scaled_df = pd.DataFrame(data=scaled_df, index=df.index, columns=df.columns)
    return scaled_df

### 군집화 평가 점수 

#### 군집화 평가 지수를 추출하기 위한 데이터프레임 만들기

In [50]:
def get_umap_df(df,embedding,labels):
    '''
    embedding: umap을 사용한 결과
    df: 기존 데이터프레임
    을 받아와서 umap 결과와 라벨을 받아오는 데이터프레임을 리턴하는 함수입니다.
    '''
    umap_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'], index=df.index)
    umap_df["label"] = labels
    return umap_df

#### 1) 노이즈 제거 클러스터링 결과 (get_without_noise_optics)
- 실루엣 점수 : 클러스터의 밀집도와 분리도를 모두 고려하여 평가하는 지표
- 범위는 -1 ~ 1
- 1에 가까울수록 좋은 군집화 결과를 나타냄

In [53]:
def silhouette(clustering_result) : 
    from sklearn.metrics import silhouette_score

    # 클러스터링 결과에서 노이즈 포인트 제외한 데이터와 클러스터 레이 추출
    X = clustering_result[['UMAP1', 'UMAP2']]
    labels = clustering_result['label']

    # 실루엣 점수 계산
    silhouette_avg = silhouette_score(X, labels)

    return silhouette_avg

#### Dunn 지수 (군집화의 일관성) :

- 군집 내의 응집도와 군집 간 분리도를 모두 고려하여 군집화의 품질을 평가
- 클러스터 내 거리와 클러스터 간 거리의 비율을 이용해 계산
- Dunn 지수: Dunn 지수가 0.7 이상 = 잘 분리된 클러스터가 형성 => 클러스터링 결과가 좋다고 평가
                    = Dunn 지수가 작다는 것은 각 클러스터 간의 거리가 가깝지 않다

In [56]:
# Dunn 지수 계산 함수
def dunn(clustering_result):
    from sklearn.metrics import davies_bouldin_score
    from sklearn.neighbors import DistanceMetric
    dist = DistanceMetric.get_metric('euclidean')
    labels = clustering_result['label']
    clusters = np.unique(labels)
    centroids = [np.mean(clustering_result.loc[labels==i, ['UMAP1', 'UMAP2']], axis=0) for i in clusters]
    max_intracluster_distances = [np.max(dist.pairwise(clustering_result.loc[labels==i, ['UMAP1', 'UMAP2']])) for i in clusters]
    min_intercluster_distances = []
    for i, c1 in enumerate(centroids):
        for j, c2 in enumerate(centroids):
            if i < j:
                dist_ij = dist.pairwise([c1, c2])[0][1]
                min_intercluster_distances.append(dist_ij)
    return np.min(min_intercluster_distances) / np.max(max_intracluster_distances)

#### Calinski-Harabasz 지수 (군집화의 분리력) :

- 군집화 결과가 실제로 잘 분리되어 있는지를 판단하는 지표
- 각 군집의 분산이 커질수록, 각 군집 간의 거리가 멀어질수록 높은 값

In [59]:
# Calinski-Harabasz 지수 계산 함수
def calinski_harabasz(clustering_result):
    from sklearn.metrics import calinski_harabasz_score
    labels = clustering_result['label']
    return calinski_harabasz_score(clustering_result[['UMAP1', 'UMAP2']], labels)